<a href="https://colab.research.google.com/github/mkvkanpur/field_theory/blob/main/Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile defs.py

import numpy as np
import math as m
from scipy.integrate import quad, romberg
from scipy.special import roots_jacobi
from scipy.optimize import fsolve


# modal KE energy (without Kolm const and epsilon)
def C(d,k):
  return((2.0/(area_sphere(d)*(d-1)))*pow(k,-d-2.0/3));

def area_sphere(d):
	return(2*np.pi**(d/2)/m.gamma(d/2.0))


def dvol_RG(d, v, z):
    G = 2*area_sphere(d-1)/((d-1)*area_sphere(d))
    return G * v**(d-1)* pow(1-z**2,(d-3)/2)

def dvol_flux(d, v):
    A = (area_sphere(d-1)/area_sphere(d))*4/(d-1)**2
    return A*v**(d-1)

# In Kraichnan's calculations
def S(d,v,w):
    x = ((v**2+w**2-1)/(2*v*w))
    y = (1+w**2-v**2)/(2*w)
    z = ((1+v**2-w**2)/(2*v))
    return v*((d-3)*z+2*z**3+(d-1)*x*y)

Overwriting defs.py


In [ ]:
from defs import *
print(area_sphere(1))

2.0


# Using Kraichnan's S(k,p,q) formula \int dp dq

In [ ]:
from defs import *

b = 1.7
d = 2
####
#### Renormalized viscosity
####

def dvol_RG_Kraichnan(d,v,w):
  x =  ((v**2+w**2-1)/(2*v*w))
  return area_sphere(d-1) * (v*w)**(d-2) * \
    (1-x**2)**((d-3)/2)

def RG_vw(d,v,w):
  numr1 = S(d,v,w)*C(d,w)
  numr2 = S(d,w,v)*C(d,v)
  denr = v**(2/3) + w**(2/3)
  return dvol_RG_Kraichnan(d,v,w)*((numr1+numr2)/denr)

lower_lim = 1
integ_RG_Kr = quad(lambda v: quad(lambda w: RG_vw(d,v,w), \
                            lower_lim,b)[0],lower_lim,b)[0]/(d-1)

print  (np.sqrt(integ_RG_Kr/(1-b**(-4/3))))

#integ_DIA = quad(lambda v: quad(lambda w: RG_vw(d,v,w),v-1,v+1)[0],1,1e3)[0]/(d-1)

#print  (np.sqrt(integ_DIA))


0.14408063504977678


# Using Kraichnan's S(k,p,q) formula \int dp dz

In [ ]:
from defs import *

import numpy as np
import math as m
from scipy.integrate import quad, romberg
from scipy.special import roots_jacobi
from scipy.optimize import fsolve

def dvol_RG(d, v, z):
    return area_sphere(d-1) * v**(d-1)* (1-z**2)**((d-3)/2)
####
#### Renormalized viscosity
####

def RG(d,v,z):
        w = np.sqrt(1 + v**2 - 2*v*z)
        F1 = S(d,v,w)*C(d,w)
        F2 = S(d,w,v)*C(d,v)
        denr = v**(2/3) + w**(2/3)
        return dvol_RG(d,v,z)*(F1+F2)/denr

def fv_RG(d,v):
        lower_lim_z = (v**2+1-b**2)/(2*v)
        upper_lim_z = v/2
        return quad(lambda z: RG(d,v,z), lower_lim_z, upper_lim_z)[0]/(d-1)

integ = quad(lambda v: fv_RG(d,v), 1, b)[0]
print(np.sqrt(integ/(1-b**(-4/3))))



0.14408063504977675


In [ ]:
### DIA ###
from defs import *

def dvol_DIA(d, v):
    return area_sphere(d-1) * v**(d-1)

def DIA_nu_integrand(d,v,z):
        w = np.sqrt(1 + v**2 - 2*v*z)
        F1 = S(d,v,w)*C(d,w)
        F2 = S(d,w,v)*C(d,v)
        denr = v**(2/3) + w**(2/3)
        return dvol_RG(d,v,z)*(F1+F2)/denr

def compute_nu_DIA(d):

    n=1000
    alpha =  beta = (d-3)/2
    [zt, wt] = roots_jacobi(n, alpha, beta, mu=False)

    def DIA_nu(d,v):
        int_z = sum(DIA_nu_integrand(d,v,zt)*wt)
        return dvol_DIA(d,v)*int_z

    integ = quad(lambda v: DIA_nu(d,v), 1, 1000000)[0]/(d-1)
    return  np.sqrt(integ)

print(compute_nu_DIA(2))

0.07353415975575094


<ipython-input-6-8adc46d6e57b>:24: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  integ = quad(lambda v: DIA_nu(d,v), 1, 1000000)[0]/(d-1)


# RG using Craya Herring formula

In [ ]:
'''
# Going for realistic 2D
b = 1.8
d = 2
lower_lim_pq = 0.8
'''
b = 1.5
d = 2
lower_lim_pq = 1

def compute_nu1(d):

    def RG_CH1(d,v,z):
        w = np.sqrt(1 + v**2 - 2*v*z)
        denr = v**(2/3) + w**(2/3)
        F1 = (1-z**2)*(v-2*z)*(2*v*z-1)*v*w**(-8/3-d)
        F2 = (1-z**2)*(1-v**2)*(2*v*z-1)*v**(-2/3-d)/w**2
        return -dvol_RG(d,v,z)*(F1+F2)/denr

    def fv_RG_CH1(d,v):
        lower_lim_z = (v**2+1-b**2)/(2*v)
        upper_lim_z = (v**2+1-lower_lim_pq**2)/(2*v)
        return quad(lambda z: RG_CH1(d,v,z), lower_lim_z, upper_lim_z)[0]

    integ_CH1 = quad(lambda v: fv_RG_CH1(d,v), 1, b)[0]
    return  np.sqrt(integ_CH1/(1-b**(-4/3)))

tollerance = 1e-5
maxiter = 150
def compute_nu2(d, nu2_left, nu2_right):
    def RG_CH2(d,nu1,nu2,v,z):
        w = np.sqrt(1 + v**2 - 2*v*z)
        F3_1 = ((1-z**2)*v**(-2/3-d))/(nu1*v**(2/3) + nu2*w**(2/3))
        F3_2 = ((1-z**2)*v**2*w**(-8/3-d))/(nu2*v**(2/3) + nu1*w**(2/3))
        return dvol_RG(d,v,z) * (F3_1+F3_2)

    def fv_RG_CH2(d,nu1,nu2,v):
        lower_lim = (v**2+1-b**2)/(2*v)
        upper_lim = (v**2+1-lower_lim_pq**2)/(2*v)
        return quad(lambda z: RG_CH2(d,nu1,nu2,v,z),lower_lim,upper_lim)[0]

    # Aim to get zero of f(nu2)
    def solve_nu2(nu1, nu2):
        integ_CH2 = quad(lambda v: fv_RG_CH2(d,nu1,nu2,v), 1, b)[0]
        return [nu2*(1-b**(-4/3)) - integ_CH2]

    nu1 = compute_nu1(d)

    x = np.zeros(maxiter); x[0] = nu2_left; x[1] =nu2_right
    for i in range(2,maxiter):
        mid = (x[i-2] + x[i-1])/2
        if (abs(mid-x[i-1]) < tollerance):
            x[i] = mid
            break
        if solve_nu2(nu1,x[i-2])[0]*solve_nu2(nu1,mid)[0] > 0:
            x[i] = x[i-1]; x[i-1] = mid
        else:
            x[i-1] = x[i-2]; x[i] = mid

    if (i > maxiter):
        print("ERROR: compute_nu1() did not converge!")
    else:
        return mid


print("d, nu1 = ", d, compute_nu1(d))
print("d, nu2 = ", d, compute_nu2(d, 0.1, 1))

def compute_nu(d):

    def RG_CH1_pCH2(d,v,z):
        w = np.sqrt(1 + v**2 - 2*v*z)
        denr = v**(2/3) + w**(2/3)
        F1 = (1-z**2)*(v-2*z)*(2*v*z-1)*v*w**(-8/3-d)
        F2 = (1-z**2)*(1-v**2)*(2*v*z-1)*v**(-2/3-d)/w**2
        F3_1 = ((1-z**2)*v**(-2/3-d))
        F3_2 = ((1-z**2)*v**2*w**(-8/3-d))
        return dvol_RG(d,v,z)*(-F1-F2+F3_1+F3_2)/denr

    def fv_RG_CH1_p_CH2(d,v):
        lower_lim = (v**2+1-b**2)/(2*v)
        upper_lim = v/2
        return quad(lambda z: RG_CH1_pCH2(d,v,z),lower_lim,upper_lim)[0]

    integ_CH1_pCH2 = quad(lambda v: fv_RG_CH1_p_CH2(d,v), 1, b)[0]
    return  np.sqrt(integ_CH1_pCH2/(1-b**(-4/3)))

print("d, nu = ", d, compute_nu(d))



d, nu1 =  2 0.09834433581222463
d, nu2 =  2 0.619207000732422
d, nu =  2 0.4845521725232924


# Energy flux and Ko: CH basis

In [ ]:
d = 3

############
# S(k,p,q)
# Original Su1u1 & Su2u2 have (1-z^2) that has been removed
# here. (1-z^2) is adjusted with (1-z^2)^{(d-3)/2}
########

def Su1u1(d,nu1,v,z):
    w = np.sqrt(1 + v**2 - 2*v*z)
    denr = nu1*(1 + v**(2/3)+w**(2/3))
    numr1 = 2*(2*v*z-1)*z*v * w**(-2)*(v*w)**(-2/3-d)
    numr2 = 2*(v-2*z)*z*v**2 * w**(-8/3-d)
    numr3 = 2*(1-v**2)*z*w**(-2) * v**(1/3-d)
    return (numr1+numr2+numr3)/denr

def Su2u2(d,nu1,nu2,v,z):
    w = np.sqrt(1 + v**2 - 2*v*z)
    denr = nu2*(1 + v**(2/3)) + nu1*w**(2/3)
    numr = v**2*w**(-8/3-d)*(v**(-2/3-d)-1)
    return numr/denr

####
# Kolmogorov's constant
# Original Su1u1 & Su2u2 have (1-z^2) that has been removed
# here. (1-z^2) is adjusted with (1-z^2)^{(d-3)/2}
########
def compute_Ko(d, v_lower_lim, nu1_eq_nu2):

    n=100
    alpha =  beta = (d-1)/2
    [rt, wt] = roots_jacobi(n, alpha, beta, mu=False)

    def flux_CH1(d,nu1,v):
        int_z = sum(Su1u1(d,nu1,v,rt)*wt)
        return int_z * dvol_flux(d,v)* np.log(1/v)

    def flux_CH2(d,nu1,nu2,v):
        int_z = sum(Su2u2(d,nu1,nu2,v,rt)*wt)
        return int_z * dvol_flux(d,v) * np.log(1/v)

    if (d==2):
        nu1 = compute_nu1(d)
        print("nu1 = ",nu1)
        integ_flux_CH1 = quad(lambda v: flux_CH1(d,nu1,v), v_lower_lim, 1)[0]
        return (abs(integ_flux_CH1))**(-2/3)
    else:   # d > 2
        if (nu1_eq_nu2):
            nu1 = nu2 = compute_nu(d)
        else:
            nu1 = compute_nu1(d)
            nu2 = compute_nu2(d, 0.3, 0.8)

        integ_flux_CH1 = quad(lambda v: flux_CH1(d,nu1,v), v_lower_lim, 1)[0]
        integ_flux_CH2 = quad(lambda v: flux_CH2(d,nu1,nu2,v), 1e-7, 1)[0]
        return (integ_flux_CH1+(d-2)*integ_flux_CH2)**(-2/3)

v_lower_lim = 1e-7
Ko = compute_Ko(d, v_lower_lim, True)
print("nu1 = nu2: Ko = ", Ko)

v_lower_lim = 0.22
Ko = compute_Ko(d, v_lower_lim, False)
print("nu1 ne nu2: Ko = ", Ko)


nu1 = nu2: Ko =  1.8524654978519122
nu1 ne nu2: Ko =  1.6308888717261192
